<a href="https://colab.research.google.com/github/dongjun0001/Dataquest/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mwclient
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1110847192),
             ('parentid', 1110846792),
             ('user', 'Grayfell'),
             ('timestamp',
              time.struct_time(tm_year=2022, tm_mon=9, tm_mday=17, tm_hour=23, tm_min=12, tm_sec=4, tm_wday=5, tm_yday=260, tm_isdst=-1)),
             ('comment',
              'Re-adding Ukraine to lead, and also adding Iran, per body of article.')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
!pip install transformers
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
  sent = sentiment_pipeline([text[:250]])[0]
  score = sent["score"]
  if sent["label"] == "NEGATIVE":
    score *= -1
  return score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.6 MB/s 
     |████████████████████████████████| 120 kB 44.4 MB/s 
     |████████████████████████████████| 6.6 MB 62.9 MB/s 


/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py:1118: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a "
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
find_sentiment("I love you")

0.9998656511306763

In [8]:
find_sentiment("I hate you")

-0.9991129040718079

In [9]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [21]:
from statistics import mean

for key in edits:
  if len(edits[key]["sentiments"]) > 0:
    edits[key]["sentiment"] = mean(edits[key]["sentiments"])
    edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
  else:
    edits[key]["sentiment"] = 0
    edits[key]["neg_sentiments"] = 0
  del edits[key]["sentiments"]

TypeError: ignored

In [14]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [15]:
edits_df

,sentiments,edit_count
2009-03-08,-0.550525,4
2009-08-05,[0.7481208443641663],1
2009-08-06,"[0.995745837688446, 0.995745837688446]",2
2009-08-14,[0.930020809173584],1
2009-10-13,"[0.5404345393180847, -0.9954361319541931]",2
...,...,...
2022-09-02,"[-0.9984153509140015, -0.9955190420150757, 0.9...",3
2022-09-06,[0.8093361854553223],1
2022-09-08,[-0.9780574440956116],1
2022-09-10,"[-0.992460310459137, 0.7481208443641663, -0.99...",3


In [16]:
edits_df.index = pd.to_datetime(edits_df.index)

In [17]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [18]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13',
               '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17',
               '2022-09-18', '2022-09-19'],
              dtype='datetime64[ns]', length=4944, freq='D')

In [19]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [20]:
edits_df

,sentiments,edit_count
2009-03-08,-0.550525,4
2009-03-09,0,0
2009-03-10,0,0
2009-03-11,0,0
2009-03-12,0,0
...,...,...
2022-09-15,0,0
2022-09-16,0,0
2022-09-17,"[-0.9955447912216187, -0.9828648567199707, -0....",4
2022-09-18,0,0


In [22]:
rolling_edits = edits_df.rolling(30).mean()

In [23]:
rolling_edits

,edit_count
2009-03-08,NaN
2009-03-09,NaN
2009-03-10,NaN
2009-03-11,NaN
2009-03-12,NaN
...,...
2022-09-15,0.533333
2022-09-16,0.433333
2022-09-17,0.566667
2022-09-18,0.566667


In [24]:
rolling_edits = rolling_edits.dropna()

In [25]:
rolling_edits

,edit_count
2009-04-06,0.133333
2009-04-07,0.000000
2009-04-08,0.000000
2009-04-09,0.000000
2009-04-10,0.000000
...,...
2022-09-15,0.533333
2022-09-16,0.433333
2022-09-17,0.566667
2022-09-18,0.566667


In [29]:
rolling_edits.to_csv("wikipedia_edits.csv")